# Modelo SVC para detección de noticias falsas

En este notebook, vamos a entrenar un modelo de clasificación utilizando **Support Vector Classification (SVC)** para predecir si una afirmación política es **verdadera** o **falsa**. El dataset proporcionado contiene afirmaciones etiquetadas junto con metadatos como el tema, el hablante, la ocupación del hablante y la afiliación política.

In [85]:
# Cargar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix


In [86]:
# Cargar los datasets limpios
train_data = pd.read_csv('/home/ines/dev/CDAW_RETO_II/LBBYs_CH2/notebooks/2_modelado/train_data_cleaned.csv')
test_data = pd.read_csv('/home/ines/dev/CDAW_RETO_II/LBBYs_CH2/notebooks/2_modelado/test_data_cleaned.csv')

In [87]:
# Funciones de preprocesamiento
def preprocess_text(text_column, max_features=1000):
    tokenizer = TfidfVectorizer(max_features=max_features)
    X_text = tokenizer.fit_transform(text_column).toarray()  # Convertir a una matriz densa
    return X_text

def preprocess_metadata(df):
    label_encoder = LabelEncoder()
    df['subject_encoded'] = label_encoder.fit_transform(df['subject'])
    df['speaker_encoded'] = label_encoder.fit_transform(df['speaker'])  # Ajuste aquí
    state_info_columns = [col for col in df.columns if col.startswith('state_info')]
    for col in state_info_columns:
        df[col] = df[col].apply(lambda x: 1 if isinstance(x, str) and len(x) > 0 else 0)
    df['state_info_encoded'] = df[state_info_columns].sum(axis=1)
    df['party_affiliation_encoded'] = label_encoder.fit_transform(df['party_affiliation'])
    metadata = df[['subject_encoded', 'speaker_encoded', 'state_info_encoded', 'party_affiliation_encoded']]
    return metadata

In [88]:
# Preprocesar texto y metadatos
X_text = preprocess_text(train_data['statement'])
X_metadata = preprocess_metadata(train_data)

In [89]:
# Escalar los metadatos
scaler = StandardScaler()
X_metadata_scaled = scaler.fit_transform(X_metadata)

# Unir texto y metadatos
X_final = np.concatenate([X_text, X_metadata_scaled], axis=1)
y = train_data['label'].values


### 1. Entrenamiento simple SVC kernel linear

In [90]:
# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo SVC kernel lineal
model_svc = SVC(kernel='linear')
model_svc.fit(X_train, y_train)

# Predicciones y evaluación
y_pred_svc = model_svc.predict(X_test)
svc_report = classification_report(y_test, y_pred_svc)
svc_conf_matrix = confusion_matrix(y_test, y_pred_svc)

# Calcular las métricas
accuracy = accuracy_score(y_test, y_pred_svc)
precision = precision_score(y_test, y_pred_svc)
recall = recall_score(y_test, y_pred_svc)
f1 = f1_score(y_test, y_pred_svc)



In [91]:
# Mostrar resultados
print(svc_report)
print(svc_conf_matrix)

              precision    recall  f1-score   support

           0       0.46      0.26      0.33       611
           1       0.69      0.84      0.76      1179

    accuracy                           0.64      1790
   macro avg       0.57      0.55      0.54      1790
weighted avg       0.61      0.64      0.61      1790

[[157 454]
 [184 995]]


### 2. Entrenamiento usando GridSearchCV (SVC con kernel linear y rbf)

In [92]:
# Definir los parámetros a probar
param_grid = {
    'C': [0.1, 1, 10, 100],  # Probar diferentes valores de C (regularización)
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],  # Probar diferentes valores de gamma
    'kernel': ['linear', 'rbf']  # Probar con kernels lineales y RBF
}

# Crear el modelo SVC
svc = SVC()

# Aplicar GridSearchCV para encontrar la mejor combinación de parámetros
grid_search = GridSearchCV(svc, param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
best_params = grid_search.best_params_
print(f"Mejores parámetros encontrados: {best_params}")

# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Evaluar el modelo con los mejores parámetros
y_pred = best_model.predict(X_test)

# Calcular métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Predicciones y evaluación
y_pred_svc = model_svc.predict(X_test)
svc_report = classification_report(y_test, y_pred_svc)
svc_conf_matrix = confusion_matrix(y_test, y_pred_svc)


Mejores parámetros encontrados: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}


In [93]:
# Mostrar resultados
print(svc_report)
print(svc_conf_matrix)

              precision    recall  f1-score   support

           0       0.46      0.26      0.33       611
           1       0.69      0.84      0.76      1179

    accuracy                           0.64      1790
   macro avg       0.57      0.55      0.54      1790
weighted avg       0.61      0.64      0.61      1790

[[157 454]
 [184 995]]
